# Fine-tuning a Model for Masked Language Modeling (MLM) Exam

In this exam, you will be tasked with performing dataset preprocessing and fine-tuning a model for a masked language modeling task. Complete each step carefully according to the instructions provided.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `bert-base-uncased` for both the model and tokenizer.
- **Dataset**: You will be using the `CUTD/math_df` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [67]:
! pip install transformers datasets evaluate

In [68]:
from datasets import load_dataset

dataset = load_dataset("CUTD/math_df", split="train[:50]")

In [69]:
dataset = dataset.train_test_split(test_size=0.2)

In [70]:
dataset['train'][0]

{'Unnamed: 0': 36,
 'text': 'A high school athletic director who helps their sibling understand grassroots sports movements'}

In [71]:
dataset['test'][1]

{'Unnamed: 0': 9,
 'text': "A native of the remote research locations who provided the retired pilot and marine biologist with valuable insight on the area's ecosystem"}

## Step 2: Load the Pretrained Model and Tokenizer

Use a pre-trained model and tokenizer for this task. Initialize both in this step.

In [72]:
from transformers import TFAutoModelForMaskedLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = TFAutoModelForMaskedLM.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


## Step 3: Preprocess the Dataset

Define a preprocessing function that tokenizes the text data and prepares the inputs for the model. Ensure that you truncate the sequences to a maximum length of 512 tokens and pad them appropriately.

**Bonus**: If you performed more comprehensive preprocessing, such as removing links, converting text to lowercase, or applying additional preprocessing techniques.

In [73]:
def preprocess(examples):
    examples['text'] = [text.lower() for text in examples['text']]
    inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
    return inputs


In [74]:
dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

## Step 4: Define Training Arguments

Set up the training configuration, including parameters like learning rate, batch size, number of epochs, and weight decay.

In [75]:
from transformers import DataCollatorForLanguageModeling
#mlm_probability: The probability with which to (randomly) mask tokens in the input, when mlm is set to True.
#The type of Tensor to return. Allowable values are “np”, “pt” and “tf”
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, return_tensors="tf")

## Step 5: Initialize the Trainer

Initialize the Trainer using the model, training arguments, and datasets (both training and evaluation).

In [76]:
tf_train_set = model.prepare_tf_dataset(
    dataset["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    dataset["test"],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

In [77]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [78]:
model.compile(optimizer=optimizer)
model.fit(x=tf_train_set ,validation_data=tf_test_set ,epochs=3)

Epoch 1/3
5/5 [==============================] - 64s 2s/step - loss: 2.9546 - val_loss: 3.0093
Epoch 2/3
5/5 [==============================] - 6s 1s/step - loss: 2.8737 - val_loss: 2.9808
Epoch 3/3
5/5 [==============================] - 7s 1s/step - loss: 2.2865 - val_loss: 2.1510


## Step 6: Fine-tune the Model

Run the training process using the initialized Trainer to fine-tune the model on the masked language modeling task.

## Step 7: Inference

Use the fine-tuned model for inference. Create a pipeline for masked language modeling and test it with a sample sentence.

In [83]:
model.save_pretrained("my-m-model")
tokenizer.save_pretrained("my-model")

('my-model/tokenizer_config.json',
 'my-model/special_tokens_map.json',
 'my-model/vocab.txt',
 'my-model/added_tokens.json',
 'my-model/tokenizer.json')

In [87]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="my-m-model", tokenizer="my-model"
)

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at my-m-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [88]:
sent = "The Milky Way is a [MASK]."
preds = mask_filler(sent)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> the milky way is a galaxy.
>>> the milky way is a milky.
>>> the milky way is a universe.
>>> the milky way is a star.
>>> the milky way is a galaxies.


In [89]:
preds

[{'score': 0.9437532424926758,
  'token': 9088,
  'token_str': 'galaxy',
  'sequence': 'the milky way is a galaxy.'},
 {'score': 0.007158227730542421,
  'token': 21582,
  'token_str': 'milky',
  'sequence': 'the milky way is a milky.'},
 {'score': 0.005128068383783102,
  'token': 5304,
  'token_str': 'universe',
  'sequence': 'the milky way is a universe.'},
 {'score': 0.0029196515679359436,
  'token': 2732,
  'token_str': 'star',
  'sequence': 'the milky way is a star.'},
 {'score': 0.002740986179560423,
  'token': 21706,
  'token_str': 'galaxies',
  'sequence': 'the milky way is a galaxies.'}]